# PRODUCTS

**A. CLEAN UP AND PREPARE PRODUCT DATA**

A.1 Load and plot

In [ ]:
%matplotlib inline
import pandas as pd

import matplotlib.pyplot as plt

import os

In [ ]:
os.getcwd()

In [ ]:
#load the data
hh_demographic = pd.read_csv('../data/dunnhumby_complete_csv/hh_demographic.csv', sep = ',')

transaction_data = pd.read_csv('../data/dunnhumby_complete_csv/transaction_data.csv', sep = ',')

product = pd.read_csv('../data/dunnhumby_complete_csv/product.csv', sep = ',')

In [ ]:
product.head(4)

First we get interested by the different departments in the commercial center. We reorder the departments and plot the number of products we can find per department. 

In [ ]:
# REORDERING PART
ordered_manufacturer= product.MANUFACTURER.astype(int).sort_values()
ordered_manufacturer= ordered_manufacturer.unique()

product['MANUFACTURER'] = pd.Categorical(product['MANUFACTURER'],
                      ordered = True,
                      categories = ordered_manufacturer)

print ('The order of the manufacturer categories is :', ordered_manufacturer)

In [ ]:
ordered_department= ['GROCERY','FROZEN GROCERY','GRO BAKERY','PASTRY',
        'SEAFOOD','SEAFOOD-PCKGD','MEAT', 'MEAT-PCKGD','MEAT-WHSE','PORK','SPIRITS','NUTRITION',
       'PRODUCE','CHEF SHOPPE','RESTAURANT','SALAD BAR','DELI', 'DAIRY DELI','DELI/SNACK BAR',
    'DRUG GM','COSMETICS','PHARMACY SUPPLY','GARDEN CENTER','FLORAL', 'TRAVEL & LEISUR', 
    'POSTAL CENTER', 'PHOTO', 'VIDEO','VIDEO RENTAL','TOYS','HOUSEWARES', 
    'KIOSK-GAS', 'ELECT &PLUMBING','AUTOMOTIVE','GM MERCH EXP','COUP/STR & MFG', 
     'CHARITABLE CONT', 'RX','PROD-WHS SALES', 'HBC','CNTRL/STORE SUP','MISC. TRANS.', 'MISC SALES TRAN',' ']

product['DEPARTMENT'] = pd.Categorical(product['DEPARTMENT'],
                      ordered = True,
                      categories = ordered_department)

print ('The order of the department categories is :', ordered_department)

In [ ]:
department= product.groupby(['DEPARTMENT']).count()

#PLOT THE NUMBER OF PRODUCTS PER DEPARTMENT

fig = plt.figure(figsize=(20,10))

# Divide the figure into a 2x1 grid, and give me the first section
ax1 = fig.add_subplot(231)

# Divide the figure into a 2x1 grid, and give me the second section
ax2 = fig.add_subplot(232)

ax3 = fig.add_subplot(233)

ax4 = fig.add_subplot(234)

ax5 = fig.add_subplot(235)

ax6 = fig.add_subplot(236)

department[(department.BRAND>0) & (department.BRAND<=2)].BRAND.sort_values().plot.bar(ax=ax1)
department[(department.BRAND>2) & (department.BRAND<=5)].BRAND.sort_values().plot.bar(ax=ax2)
department[(department.BRAND>5) & (department.BRAND<=20)].BRAND.sort_values().plot.bar(ax=ax3)
department[(department.BRAND>20) & (department.BRAND<=500)].BRAND.sort_values().plot.bar(ax=ax4)
department[(department.BRAND>500) & (department.BRAND<=4000)].BRAND.sort_values().plot.bar(ax=ax5)
department[department.BRAND>4000].BRAND.sort_values().plot.bar(ax=ax6)

fig.subplots_adjust(hspace=1)
fig.subplots_adjust(wspace=0.4)


As we can see in the previous barplots, the departments with the highest number of referenced products in this study are the grocery with almost 40000 products, and the drug GM (which is probably a drug store) with more than 30000 products. We are going to focus on these 2 mains stores in a first time.

**A. GROCERY DATA PREPROCESSING**

In this part we are interested to clean up and categorize the different products from the grocery store. We want to explain what purpose they serve (food, sanitary & hygien products, pet products). We create a special Grocery dataframe for this purpose.

In [ ]:
#we extract the grocery product data which is the main supermarket in the commercial center
Grocery= pd.DataFrame(product[product['DEPARTMENT'] == 'GROCERY'])
Grocery.head()

The only product descriptions provided by the Grocery data are few words description found in COMMODITY_DESC and SUB_COMMODITY_DESC columns. These descriptions are not standardized, no categories arise from them. In a first time, we extract and class the description words in different categories (food, sanitary, pet or other).

In [ ]:
# We create a table where all description words are extracted and separated as word 1, word 2, word 3 ... 
#from the same description line. Here we look at the words in COMMODITY_DESC

split = Grocery['COMMODITY_DESC'].str.split(' |\/').tolist()
words = pd.DataFrame.from_records(split, index= Grocery.index, columns=["w1","w2","w3","w4","w5","w6"])

# we show the number of occurrence for each first word. For example the word AIR occures 414 times as the first 
#word of a description

words.groupby('w1').count()

In [ ]:
# here are all the unique first words 
words.w1.unique()

Once we have extracted these first words of the descriptions, we assign them to the different categories.

In [ ]:
FoodW1 = ['FRZN', 'FRUIT', 'COOKIES', 'SPICES','CHEESE','COFFEE','BAKED', 'VEGETABLES','CONDIMENTS', 'BAKING',
    'PICKLE','SOUP','FROZEN', 'EGGS','REFRGRATD', 'HOT', 'COLD', 'SUGARS', 'SEAFOOD','CANNED','BEANS', 'CRACKERS',
     'BUTTER', 'WATER','MILK','TEAS', 'PASTA', 'MOLASSES','MEAT', 'SALD', 'LIQUOR', 'SNACK', 'BEERS', 'YOGURT',
    'COCOA', 'FLOUR', 'SYRUPS','OLIVES','ISOTONIC']
        
OtherW1 = ['ICE','PNT','AIR','SHORTENING','PAPER','HISPANIC','DINNER','HOUSEHOLD','FD','DRY','FLUID','BAG',
         'CONVENIENT', 'SOFT','WAREHOUSE','PWDR','MISC.','BLEACH','DOMESTIC',
       'MISC', 'COUPON', 'IMPORTED', 'RESTRICTED', 'NEW','BOTTLE']        

PetW1 = ['CAT','DOG','PET','BIRD']

SanitaryW1 = ['FACIAL','LAUNDRY','BATH','DISHWASH']


Then we reproduce it for the second and third words of the descriptions. The descriptions can have up to 6 words but the 3 first words are sufficient to determine the product category.

In [ ]:
words.w2.unique()

In [ ]:
FoodW2 = ['BREAD','SAUCES', 'VEGETABLE','FRUITS', 'WRAPS','SWEET','RELISH','PIZZA', 'DOUGH', 'CEREAL', 
        'SWEETNERS', 'JUICES', 'SNACKS','BRKFST', 'DRINKS', 'MEAT','NOODLES', 'SAUCE', 'FOOD', 'SYRUP','PIE',
        'NUTS', 'ALES','FOODS','BREAKFAST','DAIRY','TOPPINGS','POTATOES','CHICKEN', 'WINE','DIET', 'SEED', 
        'SEAFOOD']

OtherW2 =['ICE', '-', 'CONES', '&', 'BTR', 'CREAM','OIL','HOUSEWARES','MXS:DRY','NEEDS','MIX', 
          'MIXES','BN', 'TISS','MISC','BY-PRODUCTS','JCE', 'ADDITIVES','DRSNG','NOVELTIES', 'CRYSTL', 'AGE', 
          'DEPOSITS']

SanitaryW2=['CARE','CLEANG','DETERGENTS','TISSUES','TOWELS','LITTER']

In [ ]:
words.w3.unique()

In [ ]:
FoodW3 = ['JELLY', 'MILK', 'BUNS', 'VEG','DESSERTS', 'CANNED','MEAT','FOODS','PASTA',
       'PANCAKE', 'SNDWCH', 'DOUGH', 'DRNKS', 'WTR', 'GRAVY', 'DRNK',
       'MEALS']

OtherW3 =['SHELF', 'EXTRACTS','NEEDS',
       'BAGS','PKLD', 'PRODUCTS', 'GOODS', 'DNR','BKD', 'WHLSM','CARBONATED','CONC','SUPPLIES', 'ITEMS']

In [ ]:
#Initialize the categories to the grocery dataframe

Grocery['IsFood'] = False
Grocery['IsPet'] = False
Grocery['IsSanitary'] = False
Grocery['IsOther'] = False

For each word in the words dataframe we check to which category the word belongs. A boolean is turned as True in the column of their category.
We do this for each first word, then each second word, and finally for each third word. It is possible that first word belongs to a category and second word belongs to another category for example. It will be fixed when filling the grocery dataframe with the categories for each product.

In [ ]:
words['IsFoodW1'] = words.w1.isin(FoodW1)
words['IsSanitaryW1'] = words.w1.isin(SanitaryW1)
words['IsPetW1'] = words.w1.isin(PetW1)
words['IsOtherW1'] = words.w1.isin(OtherW1)

words['IsFoodW2'] = words.w2.isin(FoodW2)
words['IsSanitaryW2'] = words.w2.isin(SanitaryW2)
words['IsOtherW2'] = words.w2.isin(OtherW2)

words['IsFoodW3'] = words.w3.isin(FoodW3)
words['IsOtherW3'] = words.w3.isin(OtherW3)

In [ ]:
# We look at the words dataframe after filling with word's categories
words.head()

Finally we create a loop to assign the right categorie to each product in the grocery dataframe.

In [ ]:
# a loop to check if we have non concordance. Everything seems fine. With looking at the three first words 
# we can determine the type of product : food, pet, sanitary or other.

#uncomment to check :

#for i in words.index : 
#    if (words['IsOtherW1'].loc[i]) and (words['IsOtherW2'].loc[i]) and (words['IsOtherW3'].loc[i]) :
#        print(words.loc[i])

In [ ]:
#now we need to fill the Grocery table with the type of elements :

for i in Grocery.index : 
    if (words['IsOtherW1'].loc[i]) and (words['IsOtherW2'].loc[i]) and (words['IsOtherW3'].loc[i]) :
        Grocery.IsFood.loc[i] = False
        Grocery.IsPet.loc[i] = False
        Grocery.IsSanitary.loc[i] = False
        Grocery.IsOther.loc[i] = True
        
    elif (words['IsPetW1'].loc[i]) :
        Grocery.IsFood.loc[i] = False
        Grocery.IsPet.loc[i] = True
        Grocery.IsSanitary.loc[i] = False
        Grocery.IsOther.loc[i] = False
    
    elif (words['IsFoodW1'].loc[i]) or (words['IsFoodW2'].loc[i]) or (words['IsFoodW3'].loc[i]) :
        Grocery.IsFood.loc[i] = True
        Grocery.IsPet.loc[i] = False
        Grocery.IsSanitary.loc[i] = False
        Grocery.IsOther.loc[i] = False
        
    elif (words['IsSanitaryW1'].loc[i]) or (words['IsSanitaryW2'].loc[i]):
        Grocery.IsFood.loc[i] = False
        Grocery.IsPet.loc[i] = False
        Grocery.IsSanitary.loc[i] = True
        Grocery.IsOther.loc[i] = False

In [ ]:
Grocery.head(4)

In [ ]:
Grocery.groupby('IsFood').count()
Grocery.groupby('IsPet').count()
Grocery.groupby('IsSanitary').count()
Grocery.groupby('IsOther').count()

As shown above with groupBy function, the category with the highest number of products is the Food category, with 31886 products. We find 1887 for the pet products, and 2074 sanitary products (hygien and cleaning). Only 17 products are classified as other.

In [ ]:
Grocery.to_csv('Grocery.csv')

**B. LET'S DO IT AGAIN WITH DRUG GM**

In [ ]:
#we extract the drug gm product data which is the second main store in the commercial center
DrugGM= pd.DataFrame(product[product['DEPARTMENT'] == 'DRUG GM'])
DrugGM.head()

In [ ]:
# We create a table where all description words are extracted and separated as word 1, word 2, word 3 ... 
#from the same description line. Here we look at the words in COMMODITY_DESC

split = DrugGM['COMMODITY_DESC'].str.split(' |\/').tolist()
wordsDGM= pd.DataFrame.from_records(split, index= DrugGM.index, columns=["w1","w2","w3","w4","w5"])

# we show the number of occurrence for each first word. For example the word AIR occures 414 times as the first 
#word of a description

wordsDGM.groupby('w1').count()

In [ ]:
wordsDGM.w1.unique()

In [ ]:

SmallStuffW1 = ['ELECTRICAL','BATTERIES','CHARCOAL','HARDWARE','FILM', 
          'FIREWORKS','CANDLES' 'APPAREL', 'TOYS', 'GLASSWARE','HOSIERY','SHOE','COOKWARE',
        'GLASSES','PROPANE', 'WATCHES', 'TICKETS','CANDY','COFFEE']        

OtherW1 = ['FIRST','KITCHEN','COLD','BABY', 'FEMININE','DIETARY','SPRING', 'HAND','ORAL', 'INFANT','PLASTIC', 
           'FOOT','DISPOSIBLE', 'AUDIO', 'EYE', 'AUTOMOTIVE','OVERNIGHT', 'PERSONAL','LAWN', 'HALLOWEEN', 'SEWING',
           'COUPON', 'J-HOOKS', 'FAMILY', 'EASTER','PREPAID', 'HOME', 'IN-STORE', 'ADULT', 'DOMESTIC',
       'FALL','NATURAL', 'ETHNIC', 'CHRISTMAS', 'CONTINUITIES','PORTABLE', 'VALENTINE','MISCELLANEOUS', 'DOLLAR', 
           'FRAGRANCES', 'SPORTS', 'LONG', 'FD']

SmokeW1 = ['CIGARETTES','TOBACCO','SMOKING','CIGARS']

StationneryW1 = ['MAGAZINE','STATIONERY','NEWSPAPER', 'BOOKSTORE', 'GREETING']

PharmacyW1 = ['VITAMINS','ANALGESICS','LAXATIVES']


SanitaryW1 = ['SOAP','HAIR', 'SHAVING', 'INSECTICIDES','IRONING','DEODORANTS', 'BROOMS', 'ANTACIDS','SINUS', 
              'SUNTAN', 'DIAPERS']


In [ ]:
wordsDGM.w2.unique()

In [ ]:
SmallStuffW1 = ['GADGETS','SUPPPLIES', 'SOCKS', 'CARDS','FOODS','HOUSEWARES','ACCESSORIES', 'FOILWARE','VIDEO',
                'WIRELESS&ACCESSORIES','ELECTRIC','FURNISHINGS',  'CALCULATORS',]        

OtherW1 = ['AID', '&', 'AND', 'HBC', '-','SUMMER','FORMULA','OTHER',  'PRODUCTS', 'PHOTOFINISHING',
       'CESSATIONS', 'MISC', 'PLANNING', 'FREEZING','GOODS', 'PERSONAL', 'VISION','VALUE',
       'MEMORABLILIA', 'DISTANCE', 'WRAPS']

SmokeW1 = ['FILTERS',]

StationneryW1 = []

PharmacyW1 = ['INCONTINENCE',]


SanitaryW1 = [ 'CARE','BODY', 'HYGIENE', 'SUPPLIES','HEALTH', ]
    

In [ ]:
wordsDGM.w3.unique()